In [1]:
require(dplyr)
require(tibble)
require(tidyr)
require(reshape2)
require(ggplot2)
require(colorRamps)

Loading required package: dplyr

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: tibble
Loading required package: tidyr
Loading required package: reshape2

Attaching package: ‘reshape2’

The following object is masked from ‘package:tidyr’:

    smiths

Loading required package: ggplot2
Loading required package: colorRamps


In [2]:
potZdata <- read.table("claXYZ_pbc_p250.dat",header=TRUE)
potZdata%>% head
potZdata$Z%>%summary
str(potZdata)
potZdata$newZ <- potZdata$Z


m_lower = 57   
m_upper = 66
# Z =[56,66] is good for negative 750mV, define the boundary for bilayer and pore region maybe

#potZdata$class <- (-2+((potZdata$newZ>m_lower)+(potZdata$newZ>(m_lower+m_upper)/2)+(potZdata$newZ>m_upper)))
potZdata$class <- (-1+(potZdata$newZ> m_lower)+(potZdata$newZ> m_upper))
potZdata %>% head
#This part is to do the classification for the POT Z-axis values,
#in which, below m_lower is -1, above m_upper is +1, 
#between m_lower and m_upper is 0.

potZdata$XY_ID <- (-4+((potZdata$X>60) + (potZdata$X <120) + (potZdata$Y > 70) +(potZdata$Y <160)))
potZdata %>% head


#potZdata = potZdata[potZdata$class ==0,] # 2500frames=600ns, 212ns, NBD through Cx26 channel, 885frames.
potZdata = potZdata[potZdata$Frame<2085,] # 2500frames=600ns, 212ns, NBD through Cx26 channel, 885frames.
str(potZdata)

#potZdata$class <- (-2+((potZdata$newZ>m_lower)+(potZdata$newZ>(m_lower+m_upper)/2)+(potZdata$newZ>m_upper)))

Frame,MEASUREMENT,X,Y,Z
0,Vec_00000,177.0536,199.1626,25.40681
1,Vec_00000,160.4837,136.5072,102.85232
2,Vec_00000,161.3828,139.2563,103.74570
3,Vec_00000,163.0884,139.4902,98.64247
4,Vec_00000,164.7033,142.8944,94.31529
5,Vec_00000,163.6647,140.9599,87.79234


     Min.   1st Qu.    Median      Mean   3rd Qu.      Max. 
  0.00018  22.67220  92.63228  73.45989 116.58449 143.59898 

'data.frame':	1273572 obs. of  5 variables:
 $ Frame      : int  0 1 2 3 4 5 6 7 8 9 ...
 $ MEASUREMENT: Factor w/ 612 levels "Vec_00000","Vec_00001",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ X          : num  177 160 161 163 165 ...
 $ Y          : num  199 137 139 139 143 ...
 $ Z          : num  25.4 102.9 103.7 98.6 94.3 ...


Frame,MEASUREMENT,X,Y,Z,newZ,class
0,Vec_00000,177.0536,199.1626,25.40681,25.40681,-1
1,Vec_00000,160.4837,136.5072,102.85232,102.85232,1
2,Vec_00000,161.3828,139.2563,103.74570,103.74570,1
3,Vec_00000,163.0884,139.4902,98.64247,98.64247,1
4,Vec_00000,164.7033,142.8944,94.31529,94.31529,1
5,Vec_00000,163.6647,140.9599,87.79234,87.79234,1


Frame,MEASUREMENT,X,Y,Z,newZ,class,XY_ID
0,Vec_00000,177.0536,199.1626,25.40681,25.40681,-1,-2
1,Vec_00000,160.4837,136.5072,102.85232,102.85232,1,-1
2,Vec_00000,161.3828,139.2563,103.74570,103.74570,1,-1
3,Vec_00000,163.0884,139.4902,98.64247,98.64247,1,-1
4,Vec_00000,164.7033,142.8944,94.31529,94.31529,1,-1
5,Vec_00000,163.6647,140.9599,87.79234,87.79234,1,-1


'data.frame':	1273572 obs. of  8 variables:
 $ Frame      : int  0 1 2 3 4 5 6 7 8 9 ...
 $ MEASUREMENT: Factor w/ 612 levels "Vec_00000","Vec_00001",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ X          : num  177 160 161 163 165 ...
 $ Y          : num  199 137 139 139 143 ...
 $ Z          : num  25.4 102.9 103.7 98.6 94.3 ...
 $ newZ       : num  25.4 102.9 103.7 98.6 94.3 ...
 $ class      : num  -1 1 1 1 1 1 1 1 1 1 ...
 $ XY_ID      : num  -2 -1 -1 -1 -1 -1 -1 -1 -1 -1 ...


In [3]:
potSelect <- filter(potZdata,potZdata$class== -1)
#potSelect %>% head
#potSelect$class[1]
#write.csv(potZdata, file="potZdata_class.csv")

potUniq <- subset(potZdata,select = -c(Z,newZ))
potUniq %>% head


potSort <- potUniq %>% 
    group_by(MEASUREMENT) %>%
     #top_n(n=-1,wt=Frame) 
    mutate(class_nm=c(class[1]-1,class[c(1:(length(class)-1))])) %>%
    mutate(change=1.0*(abs(class-class_nm)>0)) %>%
    filter(change==1)
# This is to get rid of POT positions which did not change between classes, stayed in the same class.
   
potSort %>% head

Frame,MEASUREMENT,X,Y,class,XY_ID
0,Vec_00000,177.0536,199.1626,-1,-2
1,Vec_00000,160.4837,136.5072,1,-1
2,Vec_00000,161.3828,139.2563,1,-1
3,Vec_00000,163.0884,139.4902,1,-1
4,Vec_00000,164.7033,142.8944,1,-1
5,Vec_00000,163.6647,140.9599,1,-1


Frame,MEASUREMENT,X,Y,class,XY_ID,class_nm,change
0,Vec_00000,177.0536,199.1626,-1,-2,-2,1
1,Vec_00000,160.4837,136.5072,1,-1,-1,1
652,Vec_00000,138.2260,122.4232,-1,-1,1,1
653,Vec_00000,148.6199,127.6420,1,-1,-1,1
662,Vec_00000,156.3205,129.1623,-1,-1,1,1
663,Vec_00000,155.1491,127.3615,1,-1,-1,1


In [4]:
potSize <- potSort %>%
   group_by(MEASUREMENT) %>%
   summarise(n_size =length(MEASUREMENT))
potSize %>% head

MEASUREMENT,n_size
Vec_00000,94
Vec_00001,79
Vec_00002,120
Vec_00003,98
Vec_00004,77
Vec_00005,70


In [5]:
i =1
timeLot = c()
id_Lot <- c()
tot_count =0
time_interval=0.024
tot_time =0
nLev =0
nCLA=612
#& potSort$XY_ID[i] == 0
for (j in 1:nCLA)  {
    nLev = nLev + potSize$n_size[j]
    #print(potSize$n_size[j])
    # & potSort$XY_ID[i]==0
   while (i< (nLev-2)) {
       if (potSort$class[i] == 1 & potSort$class[i+1] ==0 & potSort$class[i+2]== -1  ) {
           #if (potSort$class[i] == 1 & potSort$class[i+1] ==0 & potSort$class[i+2]== -1 & potSort$XY_ID[i+1] == 0 ) {
        num =1
        tot_count = num + tot_count        
        id_Lot <- c(id_Lot,potSort$MEASUREMENT[i])         
        #print(potSort$MEASUREMENT[i+2])
        #print(paste("End Transition Event time",potSort$Frame[i+2]))
        #print(paste("Start time",potSort$Frame[i]))
        timeLot <- c(timeLot,(potSort$Frame[i+2]-potSort$Frame[i])*time_interval)
        tot_time=tot_time+(potSort$Frame[i+2]-potSort$Frame[i])*time_interval
        #print(j)
        }
    else {
        tot_count = tot_count
    }
    i=i+1
   }
    #print(nLev)
    #print(i)
   i=i+3
}
timeLot 
id_Lot 
#id_Lot
print(tot_count)
print(tot_time)
tot_time/tot_count

[1] 0.432 0.384 0.168 0.312 0.264 0.312 0.120 0.120

[1]  13  56 223 328 557 578 578 578

[1] 8
[1] 2.112


[1] 0.264

In [6]:
newLot <- timeLot[timeLot >0]
newLot
length(newLot)
sum(newLot)
sum(newLot)/length(newLot)

[1] 0.432 0.384 0.168 0.312 0.264 0.312 0.120 0.120

[1] 8

[1] 2.112

[1] 0.264